In [23]:
##!pip uninstall spacy
##!pip install spacy==3.5.3
##!python -m spacy download pt_core_news_sm
from collections import Counter
from collections import defaultdict
import pandas as pd
import unicodedata
import spacy
from collections import defaultdict

In [11]:
nlp = spacy.load("pt_core_news_sm")

def lematizar(texto):
    doc = nlp(texto)
    return ''.join(token.lemma_ for token in doc)

def remover_acentos(texto):
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )

def temEspaco(texto):
    return ' ' in texto

In [29]:
df = pd.read_excel('rawdata.xlsx', sheet_name='frases_categorias')

# Lista de palavras genéricas
palavras_genericas = ['perfume', 'fragrancia', 'cheirar', 'cheiroso', 'cheirando', 'cheiro', 'em o', 'de o', 'gostar', 'deixar','roupa','ficar','muito']
palavras_genericas.append('porque')
# Armazenar dados finais
dados_finais = []

# Processar por categoria
categorias = df['categoria'].unique()
for categoria in categorias:
    filtrado = df[df['categoria'] == categoria]
    todas_palavras = []

    for frase in filtrado['frase']:
        bruto_palavras = frase.split()
        for palavra in bruto_palavras:
            palavra_proc = lematizar(remover_acentos(palavra.lower()))
            if len(palavra_proc) > 3 and palavra_proc not in palavras_genericas and not temEspaco(palavra_proc):
                todas_palavras.append(palavra_proc)

    contagem = Counter(todas_palavras)
    top_10 = contagem.most_common(10)

    for palavra, freq in top_10:
        dados_finais.append({'categoria': categoria, 'palavra': palavra, 'quantasvezes_aparece': freq})


df_resultado = pd.DataFrame(dados_finais)
df_resultado.to_csv('resultado.csv', index=False, encoding='utf-8-sig')


In [ ]:
def classificar_frase(frase, df_resultado):
    # Pré-processamento da frase
    palavras_frase = frase.split()
    palavras_processadas = [
        lematizar(remover_acentos(p.lower()))
        for p in palavras_frase
        if len(p) > 3 and not temEspaco(p)
    ]

    # Agrupar palavras relevantes com seus pesos por categoria
    categorias_palavras_peso = defaultdict(dict)
    for _, row in df_resultado.iterrows():
        palavra = row['palavra']
        categoria = row['categoria']
        peso = row['quantasvezes_aparece']
        categorias_palavras_peso[categoria][palavra] = peso

    # Contar matches ponderados por categoria
    contagem_ponderada = {}
    for categoria, palavras_pesos in categorias_palavras_peso.items():
        peso_total = sum(
            palavras_pesos.get(p, 0) for p in palavras_processadas
        )
        contagem_ponderada[categoria] = peso_total

    # Ordenar categorias por peso total decrescente e pegar as top 3
    top_categorias = sorted(
        contagem_ponderada.items(),
        key=lambda x: x[1],
        reverse=True
    )[:3]

    return top_categorias

In [37]:
print(classificar_frase('gosteri de tudo', df_resultado))


[('No dislikes', 12), ('Perfume Characteristic', 0), ('Pleasant perfume', 0)]


In [ ]:
## Exemplo de Uso da SpaCy para Lematização

texto = "Fizemos muitas casinhas coloridas"
doc = nlp(texto)

for token in doc:
    print(f"Token: {token.text} | Lema: {token.lemma_} | POS: {token.pos_}")